In [ ]:
import os
import numpy as np
import pandas as pd
import math

In [ ]:
# # export
# def synthesize_speakerids(filelists):

#     data_dict = {}
#     for f in range(len(filelists)):
#         data_dict[filelists[f]] = pd.read_csv(filelists[f], sep = ",",index_col=0,error_bad_lines=False)
#         #pd.read_csv(filelists[f], sep = "|",header=None, error_bad_lines=False)
        
#     source_files = list(data_dict.keys())
#     nspeakers_cumulative = 0
#     speaker_offset = {}
#     for source_file in filelists:
#         data = data_dict[source_file]
#         nspeakers = len(np.unique(data.iloc[:,2]))
#         data.iloc[:,2] = data.iloc[:,2] + nspeakers_cumulative
#         data_dict[source_file] = data
#         speaker_offset[source_file] = nspeakers_cumulative
#         nspeakers_cumulative = nspeakers_cumulative + nspeakers

#     return(data_dict)  

In [ ]:
# export
import soundfile as sf


def load_filepaths_and_text(filename, split="|"):
    with open(filename, encoding='utf-8') as f:
        filepaths_and_text = [line.strip().split(split) for line in f]
    return filepaths_and_text

def synthesize_speakerids2(filelists, fix_indices_index = None):

    data_dict = {}
    data_dict_out = {}
    for f in range(len(filelists)):
        #data_dict[filelists[f]] = pd.read_csv(filelists[f], sep = ",",index_col=0,error_bad_lines=False)
            data = load_filepaths_and_text(filelists[f])
            data_dict[filelists[f]] = pd.DataFrame(data)    
        #pd.read_csv(filelists[f], sep = "|",header=None, error_bad_lines=False)
        
    source_files = list(data_dict.keys())
    
    speaker_offset = {}
    nfilelist = len(filelists)
    reserved_speakers = np.unique(data_dict[filelists[fix_indices_index]].iloc[:,2])
    
    for s in range(nfilelist):
        source_file = filelists[s]
        data = data_dict[source_file]
        if s != fix_indices_index:
            speakers = np.unique(data.iloc[:,2])
            overlap = np.where(np.isin(speakers, reserved_speakers))[0]
            reserved_speakers_temp = np.union1d(speakers, reserved_speakers)
            newindices = np.setdiff1d(list(range(len(reserved_speakers) + len(speakers))), reserved_speakers_temp)[:len(overlap)]
            for o in range(len(overlap)):
                data.iloc[np.where(data.iloc[:,2] == overlap[o])[0] ,2] = newindices[o]

            data_dict_out[source_file] = data
            speakers = np.unique(data.iloc[:,2])
            reserved_speakers = np.union1d(speakers, reserved_speakers)
            #print(speakers,reserved_speakers)
        else:
            data_dict_out[source_file] = data
    return(data_dict_out)  

In [ ]:
metalist_dir = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/metadata_collections'
metalist_files = os.listdir(metalist_dir)
print(metalist_files)
train_ratios = np.ones(4)*1.

['lj7_processed_file.npy', 'libritts_processed_file.npy', 'uberduck_processed_files.npy', 'vctk_processed_file.npy']


In [ ]:
filelists=np.asarray([])
#files = np.asarray([])
for r in range(4):
    files = np.load(metalist_dir +'/'+ metalist_files[r],allow_pickle=True)
    filelist = np.asarray([])
    
    if files.ndim > 0:
        nfiles = files.shape[0]
        for s in range(nfiles):
            filelist = np.append(filelist, files[s])
    else: 
        filelist = files
    filelists = np.append(filelists, filelist)

In [ ]:
print(filelists)
dd = synthesize_speakerids2(filelists,1)

['/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/processed_metadata/lj7all_processed.txt'
 '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/processed_metadata/librittsall_processed.txt'
 '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/processed_metadata/eminem_all_processed.txt'
 '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/processed_metadata/ben-shapiro_all_processed.txt'
 '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/processed_metadata/jay-z_all_processed.txt'
 '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/processed_metadata/vctkall_processed.txt']


/opt/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:565: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)


In [ ]:
ad = list(dd.values())
#ad2 = [ad[i] for i in [0,2,3]]
#ad2 = [ad[i] for i in [2,3,4]]
#ad2 = [ad[i] for i in [0]]
ad2 = [ad[i] for i in [2]]
alldata = pd.concat(ad2)

In [ ]:
train_ind = np.random.choice(alldata.shape[0], math.floor(alldata.shape[0]*.8), replace = False)
val_ind = np.setdiff1d(list(range(alldata.shape[0])), train_ind)

In [ ]:
alldata.iloc[train_ind,:].to_csv('/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/todos_train.txt', header = False,sep = '|', index=False)

In [ ]:
alldata.iloc[val_ind].to_csv('/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/todos_val.txt', header = False,sep = '|', index=False)

In [ ]:
#audiopaths_and_text = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/todos_train.txt'
audiopaths_and_text = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/processed_metadata/eminem_all_processed.txt'
audiopaths_and_text = load_filepaths_and_text(audiopaths_and_text)
#asdf = [x[2] for x in audiopaths_and_text]


In [ ]:
import librosa

In [ ]:
librosa.load(audiopaths_and_text[4][0])

(array([ 0.00335953,  0.00497484,  0.00431533, ..., -0.00397925,
        -0.00451155,  0.        ], dtype=float32),
 22050)

In [ ]:
sf.read(audiopaths_and_text[4][0])

(array([ 0.00469971,  0.00460815,  0.00466919, ..., -0.00439453,
        -0.00415039, -0.00308228]),
 48000)

In [ ]:
audiopaths_and_text[5][0]

'/mnt/disks/uberduck-experiments-v0/data/LJSpeech-1.1/wavs/LJ010-0305.wav'

In [ ]:
filename = audiopaths_and_text

In [ ]:
with open(filename, encoding='utf-8') as f:
    filepaths_and_text = [line for line in f]

In [ ]:
### source_files = [experiment_filelist_dir + 'eminem_all.txt',
#             '/mnt/disk/uberduck-experiments-v0/data/uberduck/kanye-rap/all.txt',
#             '/mnt/disk/uberduck-experiments-v0/data/uberduck/alex-trebek/all.txt',
#             '/mnt/disk/uberduck-experiments-v0/data/uberduck/ben-shapiro/all.txt',
#             '/mnt/disk/uberduck-experiments-v0/data/uberduck/michael-rosen/all.txt',
#             '/mnt/disk/uberduck-experiments-v0/data/uberduck/steve-harvey/all.txt',
#             '/mnt/disk/uberduck-experiments-v0/data/uberduck/LJSpeech-1.1/metadata.csv',
#             '/mnt/disk/uberduck-experiments-v0/data/uberduck/LibriTTS/metadata_mellotron.txt',
#             '/mnt/disk/uberduck-experiments-v0/data/uberduck/vctk/metadata.txt']

# synthesize_speakerids(source_files) #need to change
# reindex()

# train,val  = get_testval_split()

# TTSModel('mellotron')